In [1]:
import os
os. chdir('../')

In [ ]:
! python wav2vec2_training.py

In [2]:
from constants.mir_constants import WAV2VEC2_ARGS
from training.wav2vec2_finetune import Wav2Vec2SpeechRecognition, SpeechRecognitionData
import json
from dataclasses import dataclass, asdict
import pandas as pd
from jiwer import wer

In [3]:
# import wandb
# from pytorch_lightning.loggers import WandbLogger

In [4]:
# run = wandb.init(
#     project="song lyrics transcription",
#     notes="",
#     tags=["AdamW", "Finetune Wav2vec2"]

In [5]:
# wandb_logger = WandbLogger()
# trainer = Trainer(logger=wandb_logger)

In [6]:
# wandb.config = asdict(WAV2VEC2_ARGS)

In [7]:

#print hyperparameters used for training this model into wandb
print(json.dumps(asdict(WAV2VEC2_ARGS), indent = 4))

speech_recognition_task = Wav2Vec2SpeechRecognition(wav2vec2_args=WAV2VEC2_ARGS)

#print shape of data in console. Will be visible in wandb logger
print(json.dumps(speech_recognition_task.shape(), indent = 4))


{
    "TRAIN_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_song_metadata_en_demucs_cleaned.csv",
    "TEST_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_en_demucs_cleaned.csv",
    "MODEL_BACKBONE": "facebook/wav2vec2-large-960h-lv60-self",
    "BATCH_SIZE": 1,
    "NUM_EPOCHS": 15,
    "MODEL_SAVE_PATH": "/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt",
    "FINETUNE_STRATEGY": "no_freeze_deepspeed",
    "LR_SCHEDULER": "reduce_on_plateau_schedule"
}


wandb: Currently logged in as: greeshmasmenon (songslyricstranscription). Use `wandb login --relogin` to force relogin


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


{
    "training": [
        4342,
        8
    ],
    "validation": [
        1086,
        8
    ]
}


In [8]:
speech_recognition_task.finetune()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | model         | Wav2Vec2ForCTC | 315 M 
1 | train_metrics | ModuleDict     | 0     
2 | val_metrics   | ModuleDict     | 0     
3 | test_metrics  | ModuleDict     | 0     
-------------------------------------------------
315 M     Trainable params
0         Non-trainable params
315 M     Total params
630.943   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [9]:

inference_files_lst = ["/home/users/gmenon/workspace/songsLyricsGenerator/src/notebooks/separated/mdx_extra/ad887cbfa84749e5a9789f303e2f5c30/vocals.wav",
                  "/home/users/gmenon/dali/DALI_v1.0/audio/wav_clips/ad887cbfa84749e5a9789f303e2f5c30.wav"]

inference_predictions = speech_recognition_task.inference(inference_files=inference_files_lst,
                   batch_size=4,
                   model_path=WAV2VEC2_ARGS.MODEL_SAVE_PATH,
                  wav2vec2_trainer = speech_recognition_task.wav2vec2_trainer)


Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4342it [00:00, ?it/s]

In [10]:
print(f"Inference Predictions = {inference_predictions}")

print("Starting Validation dataset Word Error Rate Metric Calculation")


Inference Predictions = [['I AM NOT THE ONE I YU TO CLA', 'I AM  O I A LA']]
Starting Validation dataset Word Error Rate Metric Calculation


In [11]:
test_data = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)

test_datamodule = SpeechRecognitionData.from_files(
    predict_files=list(test_data["consolidated_file_path"]), 
    batch_size=1)


In [12]:
finetuned_predictions = speech_recognition_task.wav2vec2_trainer.predict(speech_recognition_task.wav2vec2_model, 
                                                datamodule=test_datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 4342it [00:00, ?it/s]

In [13]:
test_data["finetuned_predictions"] = finetuned_predictions

In [14]:

finetuned_pred_transformed = []
for predictions in finetuned_predictions:
    finetuned_pred_transformed.append(predictions[0])

In [15]:
reference = list(test_data["transcription_capitalized"])

In [16]:
hypothesis = finetuned_pred_transformed


In [17]:
error = wer(reference, hypothesis)
print(f"Word Error Rate = {error}")

Word Error Rate = 0.8220123501634581


-----